In [53]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API


filters = [
    'areaType=nation',
    'areaName=England'
    ]

structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "cumDeaths28DaysByDeathDateRate"  
    }

api = Cov19API(filters=filters, structure=structure)
timeseries=api.get_json()

with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)
    
with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)
    
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")


datalist = data['data']
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()
startdate = parse_date(dates[0])
enddate = parse_date(dates[-1])
index=pd.date_range(startdate, enddate, freq='D')
timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])


for entry in datalist:
    date=parse_date(entry['date'])
    for column in ['cases', 'hospital', 'deaths']:
        if pd.isna(timeseriesdf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            timeseriesdf.loc[date, column]=value
            
timeseriesdf.fillna(0.0, inplace=True)



def wrangle_data(rawdata):
    datalist = rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    starting = parse_date(dates[0])
    ending = parse_date(dates[-1])
    index=pd.date_range(starting, ending, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    cases=hospital=deaths=0.0
    for i in range(0,100):
        timeseriesdf.loc[i,'cases']=cases
        timeseriesdf.loc[i,'hospital']=hospital
        timeseriesdf.loc[i, 'deaths'] = deaths
        cases+=np.random.randn()
        hospital+=2*np.random.randn()
        deaths+=3*np.random.randn()
    return timeseriesdf


def access_api():
    filters = [
    'areaType=nation',
    'areaName=England'
    ]

    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "cumDeaths28DaysByDeathDateRate"  
    }

    api = Cov19API(filters = filters, structure = structure)
    timeseriesOnRefresh = api.get_json()
    apibutton.icon = "check"
    apibutton.disabled = True
    return timeseriesOnRefresh

def api_button_callback(button):
    apidata=access_api()
    global timeseriesdf
    timeseriesdf=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon = "check"
    apibutton.disabled = True
    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='success', 
    tooltip='Click to download current Public Health England data',
    icon='folder'
    )

apibutton.on_click(api_button_callback)
print('NOTE: By clicking this button a random data will be set to api to demonstrate a refreshed data in the subjects')
display(apibutton)

def plot_random_walk(walk):
    timeseriesdf[walk].plot()
    plt.show()
    
whichwalk=wdg.Dropdown(
    options=['cases', 'hospital', 'deaths'],
    value='cases',
    description='subject: ',
    disabled=False,
    )

def refresh_graph():
    current=whichwalk.value
    if current==whichwalk.options[0]:
        other=whichwalk.options[1]
    else:
        other=whichwalk.options[0]
    whichwalk.value=other
    whichwalk.value=current
    
    
graph=wdg.interactive_output(plot_random_walk, {'walk': whichwalk})
print('NOTE: this graph is single tasking meaning it can show one subject at the time if refresh button clicked refreshes automatically')
display(whichwalk, graph)

series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
    )

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
    )

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

ctrls=wdg.VBox([series, scale])

form=wdg.HBox([graph, ctrls])
print('NOTE: this graph is multiple tasking which can show multiple subjects at the same time with selectable modes')
display(form)

NOTE: By clicking this button a random data will be set to api to demonstrate a refreshed data in the subjects


Button(button_style='success', description='Refresh data', icon='folder', style=ButtonStyle(), tooltip='Click …

NOTE: this graph is single tasking meaning it can show one subject at the time if refresh button clicked refreshes automatically


Dropdown(description='subject: ', options=('cases', 'hospital', 'deaths'), value='cases')

Output()

NOTE: this graph is multiple tasking which can show multiple subjects at the same time with selectable modes
